# stacgee: Google Earth Engine with STAC Metadata Tutorial

**stacgee** (rewritten from `geestac`) provides a powerful way to interact with the Google Earth Engine STAC catalog and hundreds of other public STAC catalogs. It allows you to initialize GEE objects while retaining access to the rich metadata stored in the STAC catalog (like bitmask definitions, class names, and scaling factors).

Author: Pulakesh Pradhan  
Repository: [https://github.com/pulakeshpradhan/stacgee](https://github.com/pulakeshpradhan/stacgee)

## 1. Installation and Setup

First, ensure you have the necessary libraries installed. You will need `earthengine-api`, `geemap`, and `stacgee`.

### Install from PyPI or GitHub
```bash
pip install earthengine-api stacgee geemap
```

### Install from locally built .whl file
If you have built the `.whl` file locally, you can install it as follows (adjust the path to your `dist` folder):

In [ ]:
import os
# Install the locally built wheel if it exists in the parent dist folder
if os.path.exists('../dist/stacgee-0.0.1-py3-none-any.whl'):
    !pip install ../dist/stacgee-0.0.1-py3-none-any.whl

In [ ]:
import ee
import stacgee
import geemap
import json

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='spatialgeography')

## 2. Dynamic World: Accessing Land Cover Classes

Dynamic World is a 10m global land cover product. In standard GEE, finding the class names and colors requires checking the documentation. With `stacgee`, it's available directly in the object metadata.

In [ ]:
# Initialize like a standard GEE ImageCollection
dw = stacgee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")

print(f"Asset ID: {dw.assetId}")
print(f"Start Date: {dw.start_date}")

# Access class information from the 'label' band
label_band = dw.bands.label
print("\nDynamic World Classes:")
for category in label_band.class_info:
    print(f"  Value {category.value}: {category.description} (Color: {category.color})")

## 3. Sentinel-2: Understanding QA Bitmasks

Quality assessment bands (like `QA60`) often store multiple flags in individual bits. `stacgee` decodes these bitmasks for you.

In [ ]:
s2 = stacgee.ImageCollection("COPERNICUS/S2_SR")
qa = s2.bands.QA60

if hasattr(qa, "bitmask"):
    print("Sentinel-2 QA60 Bitmask Definition:")
    print(json.dumps(qa.bitmask.to_dict(), indent=2))

## 4. GEE Processing via Proxy

`stacgee` objects act as proxies to the underlying GEE objects. You can call native GEE methods like `.filter()`, `.median()`, or `.clip()` directly on them.

In [ ]:
# Create a region of interest
roi = ee.Geometry.Point([77.2090, 28.6139]).buffer(5000)

# filterDate and filterBounds are GEE methods!
# They work here because dw proxies to the underlying ImageCollection.
filtered_dw = dw.filterBounds(roi).filterDate('2023-01-01', '2023-06-01')

print(f"Images found: {filtered_dw.size().getInfo()}")

# Create a composite using native GEE computation
median_composite = filtered_dw.select('label').mode()

# Visualize
Map = geemap.Map()
Map.centerObject(roi, 12)
Map.addLayer(median_composite, {'min': 0, 'max': 8, 'palette': ['419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc351', 'c4281b', 'a59b8f', 'b39fe1']}, 'Dynamic World Mode')
Map

## 5. MODIS: Scaling and Offsets

Many GEE datasets require multiplying raw values by a scale factor. `stacgee` makes these factors easily accessible from the STAC definition.

In [ ]:
modis = stacgee.ImageCollection("MODIS/061/MOD11A1")
lst_day = modis.bands.LST_Day_1km

print(f"Band: {lst_day.name}")
print(f"Scale Factor: {lst_day.multiplier}")
print(f"Offset: {lst_day.offset}")

# Applying the scale directly in the compute expression
# We use the multiplier from STAC metadata to apply it in the GEE cloud!
actual_temp = modis.select('LST_Day_1km').first().multiply(lst_day.multiplier).add(lst_day.offset)
print("Successfully applied STAC-derived scale and offset to GEE Image.")

## 6. Full Analysis Workflow: Leveraging Metadata for Cloud Computing

In this example, we will calculate the Normalized Difference Vegetation Index (NDVI) for Landsat 9. We will use `stacgee` to find the correct band names and their scaling factors, then perform the math using Earth Engine's cloud infrastructure.

In [ ]:
# 1. Load Landsat 9 Collection 2 Level 2 via stacgee
l9 = stacgee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# 2. Use STAC metadata to get band info
nir_band = l9.bands.SR_B5
red_band = l9.bands.SR_B4

print(f"NIR Multiplier: {nir_band.multiplier}")
print(f"Red Multiplier: {red_band.multiplier}")
print(f"Offset: {nir_band.offset}")

# 3. Create a workflow using native EE methods on stacgee proxy
roi = ee.Geometry.Point([77.2090, 28.6139]).buffer(20000)

# Masking clouds using bitmask info from STAC (QA_PIXEL)
qa_band = l9.bands.QA_PIXEL
cloud_bit = qa_band.bitmask.parts.cloud.first
dilated_cloud_bit = qa_band.bitmask.parts.dilated_cloud.first

def mask_clouds(image):
    qa = image.select('QA_PIXEL')
    # Create a mask where bits 1 and 3 are 0 (no cloud, no dilated cloud)
    mask = qa.bitwiseAnd(1 << cloud_bit).eq(0).And(
           qa.bitwiseAnd(1 << dilated_cloud_bit).eq(0))
    return image.updateMask(mask)

# 4. Apply analysis in GEE Cloud
ndvi_median = (l9.filterBounds(roi)
               .filterDate('2023-01-01', '2023-12-31')
               .map(mask_clouds)
               .median()
               .clip(roi))

# Scaling the bands before NDVI
nir_scaled = ndvi_median.select('SR_B5').multiply(nir_band.multiplier).add(nir_band.offset)
red_scaled = ndvi_median.select('SR_B4').multiply(red_band.multiplier).add(red_band.offset)

ndvi = nir_scaled.subtract(red_scaled).divide(nir_scaled.add(red_scaled)).rename('NDVI')

print("NDVI calculation complete using STAC metadata for logic!")

# 5. Visualize result
Map = geemap.Map()
Map.centerObject(roi, 11)
Map.addLayer(ndvi, {'min': 0, 'max': 0.8, 'palette': ['white', 'green']}, 'Landsat 9 NDVI')
Map

## 7. Accessing Global STAC Catalogs via STAC Index

Beyond GEE, `stacgee` can explore hundreds of public STAC catalogs via **STAC Index**.

In [ ]:
from stacgee import stacindex

# List some available catalogs
print("Available Global Catalogs (Sample):")
print(list(stacindex.children.keys())[:10])

# Access Microsoft Planetary Computer
pc = stacindex.microsoft_pc()
print(f"\nCatalog: {pc.title}")
print(f"Description: {pc.description}")